In [1]:
import inspect

def is_fitted(model):
    """Checks if model object has any attributes ending with an underscore"""
    return 0 < len( [k for k,v in inspect.getmembers(model) if k.endswith('_') and not k.startswith('__')] )

In [2]:
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.preprocessing.data import StandardScaler
from sklearn.naive_bayes import GaussianNB
import json
from airflow.operators.python_operator import PythonOperator
from collections import Counter, OrderedDict
import pandas

a = OrderedDict({})
isinstance(a, dict)

True

In [3]:
def import_dynamically(obj):
    import_statement = None
    try:
        import_statement = "from {} import {}\n".format(
                                                      obj.__module__,
                                                      obj.__name__)
    except Exception as e:
            print("Dynamic import failed. Trying simpler import.")
            import_statement = 'import {}\n'.format(obj.__name__)
    
    return import_statement
    
import_dynamically(GaussianNB)
import_dynamically(LogisticRegression)

'from sklearn.linear_model.logistic import LogisticRegression\n'

In [4]:
# cd /mnt/c/Users/<ubuntu.username>/Pictures
import pandas as pd
pd.get_dummies.__module__
import pandas
from scipy.stats import *
from pandas.core.reshape.reshape import get_dummies
import inspect
inspect.__dict__

{'__name__': 'inspect',
 '__doc__': "Get useful information from live Python objects.\n\nThis module encapsulates the interface provided by the internal special\nattributes (co_*, im_*, tb_*, etc.) in a friendlier fashion.\nIt also provides some help for examining source code and class layout.\n\nHere are some of the useful functions provided by this module:\n\n    ismodule(), isclass(), ismethod(), isfunction(), isgeneratorfunction(),\n        isgenerator(), istraceback(), isframe(), iscode(), isbuiltin(),\n        isroutine() - check object types\n    getmembers() - get members of an object that satisfy a given condition\n\n    getfile(), getsourcefile(), getsource() - find an object's source code\n    getdoc(), getcomments() - get documentation on an object\n    getmodule() - determine the module that an object came from\n    getclasstree() - arrange classes so as to represent their hierarchy\n\n    getargvalues(), getcallargs() - get info about function arguments\n    getfullargspe

In [5]:
# import_dynamically(find_missing_data)
# import_dynamically(normalize_values)
# t = DagLayer({})
# isinstance(t, DagLayer)

In [6]:
import sys
sys.path.append("../repos/WMP_ML_MVP/")

# from wmp_ml.static.missing_data import *
# from wmp_ml.static.feature_engineering import *

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

# Create an instance of Logistic Regression Classifier and fit the data.
logreg.fit(X, Y)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

In [8]:
# acc = DagLayer({accuracy_score: None, f1_score: None, precision_score: None, confusion_matrix: None, 
#                    classification_report: None, recall_score :None})

# winsorize_layer = DagLayer({wmp_winsorize: {'limits' : [0.05, 0.05]}})
def flatten_lists(l, sublist, rec_list):

    for item in l:
        if isinstance(item, list):
            flatten_lists(item, sublist, True)
        else:
            sublist.append(item)
            
    return sublist
    
def flatten_lists_master(l):
    return flatten_lists(l, [], False)

ll = [['at',['vb', ['ddd >> t34','cb'], 'd'],'fvv'], ['a','b','vv']]

res = flatten_lists_master(ll)
str(res).replace("'","")

'[at, vb, ddd >> t34, cb, d, fvv, a, b, vv]'

In [9]:
'''
dictionary = something belongs to something else, is kwargs, or has certain order
list = collection of several parallel operations

blocks -> first level keys (THESE HAVE SET ORDER)
Below this ^^^ -> The order you provide the code will matter
    - dict = matters
    - list = does not matter

'''
import sys
import os
#print(os.getcwd())
sys.path.append("../")
from sklearn.preprocessing import *
from sklearn.model_selection import *
from scipy.stats import *
import pandas as pd
from sklearn.metrics import *
from wmp_ml.static.splitting import *
from wmp_ml.static.preprocessing import *
from wmp_ml.static.feature_engineering import *
from wmp_ml.static.eda import *
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from wmp_ml.dag.layers import DagLayer

kwargs = \
{   
     'modeling': { 
                    'models': DagLayer({'LOG': {LogisticRegression: None}, 
                               'GNB': {GaussianNB: None}}),              
                 },
 
     'data_sources': [DagLayer({'https://raw.githubusercontent.com/SamShowalter/WMP_training/01_EDA/data/IBM_Employee_Attrition.csv': {pd.read_csv:None}})],

#Expects flat dataset
    'splitting': [
                        DagLayer({'split': {auto_train_test_split  : {'target': "Attrition",
                                                                        'test_ratio': 0.2,
                                                                         'random_state': 42}}}),
                 ],
 
     
     #Expects flat dataset
        'preprocessing': { 
                        
                        'cleansing': [   
                                              DagLayer({'winsorize': {wmp_winsorize: {'limits' : [0.05, 0.05],
                                                                                      'col_names': ['MonthlyRate',
                                                                                                    'NumCompaniesWorked',
                                                                                                   'PercentSalaryHike']}}}),
                                              DagLayer({'median_impute': {wmp_impute : {'method': "median"}}})
                                     ]
                        
                     },
    #Expects flat dataset
    'feature_engineering': {
                            'transformations': DagLayer({
#                                     ('Department', 'EducationField', 'JobRole', "MaritalStatus"): {pd.get_dummies: None},
#                                     'BusinessTravel': {create_ordinal_df: {'ordinal_dict':{"Non-Travel" :0, 
#                                                                                             "Travel_Rarely":1, 
#                                                                                             "Travel_Frequently": 2},
#                                                                           'tag': None}},
                                    ('Attrition', 'Gender', 'OverTime') : {convert_boolean_df: 
                                                                           {'boolean_names_and_values':
                                                                            {"Attrition": ["Yes", "No"],
                                                                            "Gender": ["Male", "Female"],
                                                                            "OverTime": ["Yes", "No"]}, 
                                                                            'tag' : None}},
#                                     ('Age', 'DistanceFromHome', 'EmployeeCount', "EnvironmentSatisfaction",
#                                     'JobInvolvement', 'JobSatisfaction', "MonthlyIncome",
#                                     'NumCompaniesWorked', 'PercentSalaryHike', "PerformanceRating",
#                                     'RelationshipSatisfaction', 'StandardHours', 'TotalWorkingYears',
#                                     'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
#                                     'YearsInCurrentRole', 'YearsSinceLastPromotion',
#                                     'YearsWithCurrManager') : 
                                    "Age": {normalize_values: {'transformer': StandardScaler, 
                                                                                   'tag': None}},
                                    'MonthlyRate': {wmp_linear_transformation: {'method': boxcox},
                                                    normalize_values: {'transformer': StandardScaler,
                                                                       'tag': None}},
                               })
                            
                           },
 
#      #Expects predictions
#      'evaluation':{'metrics' : DagLayer({'acc': {accuracy_score: None}, 
#                                         'f1': {f1_score: None}, 
#                                         'precision': {precision_score: None},
#                                         'conf_matrix': {confusion_matrix: None}, 
#                                         'class_report': {classification_report: None},
#                                         'recall': {recall_score :None}})
#                   }
   
}

In [10]:
def convert_boolean_df(data, boolean_names_and_values, tag = "_bool"):

    boolean_df = data

    if not isinstance(boolean_df, pd.Series):
        raise ValueError("Input data has more than one column")

    if tag != None:
        tag = "_" + tag
    else:
        tag = ""

    yes = boolean_names_and_values[data.name][0]
    no = boolean_names_and_values[data.name][1]

    boolean_df = boolean_df.replace({yes: 1,
                                      no: 0}, 
                                      inplace = False)

    boolean_df.name = boolean_df.name + tag


    return boolean_df

In [11]:
#df.columns

In [12]:
# convert_boolean_df(df.Attrition, **{'boolean_names_and_values':{"Attrition": ["Yes", "No"],
# "Gender": ["Male", "Female"],
# "OverTime": ["Yes", "No"]}, 'tag': None})

In [13]:
import inspect
import traceback
def import_dynamically(obj):
    import_statement = None
    try:
        import_statement = "from {} import {}\n".format(
                                                      obj.__module__,
                                                      obj.__name__)
    except Exception as e:
            print("Dynamic import failed. Trying simpler import.")
            import_statement = 'import {}\n'.format(obj.__name__)
    
    return import_statement

def rec_imports(config_section, import_list, import_check):
    
    try:
        config_keys = None
        if isinstance(config_section, DagLayer):
            config_section = config_section.layer_config
            config_keys = config_section.keys()

        if isinstance(config_section, set):
            config_keys = config_section

        else:
            config_keys = config_section.keys()

        for key in config_keys:

            if (any([inspect.isclass(key),
                   inspect.isfunction(key),
                   inspect.ismodule(key)]) 
                   and key not in import_check
                   and not isinstance(key, DagLayer)):
                import_list.append(import_dynamically(key))
                import_check.add(key)

            elif (isinstance(key, set) or 
                isinstance(key, DagLayer)):
                rec_imports(key, import_list, import_check)
            
            else:
                rec_imports(config_section[key], import_list, import_check)
                
    
    except Exception as e:
        print(traceback.print_exc())
        return ""
            

def write_imports(config):
    import_list = []
    import_check = set()
    for key in config:
        if isinstance(config[key], dict):
            rec_imports(config[key], import_list, import_check)
        if isinstance(config[key], DagLayer):
            rec_imports(config[key].layer_config, import_list, import_check)
    print("\n\n")      
    return "".join([item for item in import_list if item != ""])

print(write_imports(kwargs))

Traceback (most recent call last):
  File "<ipython-input-13-d45316729d62>", line 20, in rec_imports
    config_section = config_section.layer_config
AttributeError: 'DagLayer' object has no attribute 'layer_config'


None


Traceback (most recent call last):
  File "<ipython-input-13-d45316729d62>", line 27, in rec_imports
    config_keys = config_section.keys()
AttributeError: 'list' object has no attribute 'keys'


None


Traceback (most recent call last):
  File "<ipython-input-13-d45316729d62>", line 20, in rec_imports
    config_section = config_section.layer_config
AttributeError: 'DagLayer' object has no attribute 'layer_config'


None






In [14]:
d = {}
d['test'] = ['something']
d.setdefault('test', []).append('new_something')
d.setdefault('astest', []).append('new_something')
d

{'test': ['something', 'new_something'], 'astest': ['new_something']}

In [15]:
# kwarg_dict = collections.OrderedDict(kwargs)
# # for item in kwarg_dict.keys():
# #     print(kwarg_dict[item])

# #test = collections.OrderedDict(kwarg_dict['preprocessing'])
# for key in kwarg_dict['preprocessing']:
#     if isinstance(key, tuple):
#         for item in key:
#             print(item)
#     else:
#         print(kwarg_dict['preprocessing'][key])
        
l = []
l.append({'asf': 'asd'})
l

[{'asf': 'asd'}]

In [16]:
def abc(a, b = 2, c = 3):
    print(a, b, c)

test = {'b': 1, 'c': 1}
    
abc(2, **test)
abc(1, **{})

def a(b ,a):
    print(b)
    print(a)

a(a = 4, b = 5)

2 1 1
1 2 3
5
4


In [17]:
def normalize_values(data, transformer, tag = None):
    
    if not isinstance(data, pd.Series):
        raise ValueError("Input data has more than one column")


    # data = data.values.reshape(-1,1)

    # data = transformer().fit_transform(data)
    
    if tag != None:
        tag = "_" + tag
    else:
        tag = ""

    data = (data - data.mean()) / data.std()
    # normalized = pd.Series(data)
    # normalized.name = normalized.name + tag

    return data

In [18]:
#normalize_values(df.MonthlyRate, StandardScaler, tag = None)

In [19]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/SamShowalter/WMP_training/01_EDA/data/IBM_Employee_Attrition.csv')

In [20]:
def wmp_impute(data, method = "median"):

    nulls = data.isnull().sum()
    null_names = nulls[nulls > 0].index.tolist()
    print(nulls)
    for name in null_names:
        if method == "median":
            data[name] = data[name].fillna(data[name].median(), inplace = False)

    return data

In [21]:
df = wmp_impute(df)
wmp_impute(df)

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   1
Department                  0
DistanceFromHome            2
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              1
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              1
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               2
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102.0,Sales,1.0,2,Life Sciences,1,1.0,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279.0,Research & Development,8.0,1,Life Sciences,1,2.0,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373.0,Research & Development,2.0,2,Other,1,4.0,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392.0,Research & Development,3.0,4,Life Sciences,1,5.0,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591.0,Research & Development,2.0,1,Medical,1,7.0,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884.0,Research & Development,23.0,2,Medical,1,2061.0,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613.0,Research & Development,6.0,1,Medical,1,2062.0,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155.0,Research & Development,4.0,3,Life Sciences,1,2064.0,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023.0,Sales,2.0,3,Medical,1,2065.0,...,4,80,0,17,3,2,9,6,0,8


In [22]:
#Provided function for detecting outliers
def detect_outliers(col_names, data, std_thresh = 6):
    
    for column in col_names:
        
        #Create z_score proxy for each column
        data['z_score'] = np.absolute(zscore(data[column]))
        
        #Determine if there are outliers, as defined by z_score threshold
        outliers = data.loc[data.z_score > std_thresh, [column, 'z_score']]
        
        #If there are no outliers
        if outliers.shape[0] == 0:
            print("No outliers for column {} at threshold of {} stdevs".format(column, std_thresh))
        
        #If there are outliers
        else:
            print("\n {} outlier(s) found for column {} at threshold of {} stdevs. See below".format(outliers.shape[0],
                                                                                        column, std_thresh))
            print("\n")
            print(outliers)
            print("\n")
        
        #Drop z_score from data
        data.drop('z_score', axis = 1, inplace = True)

In [23]:
#LogisticRegression.__dict__

In [24]:
import pandas as pd
import os
import sys
sys.path.append("../wmp_ml/static")
sys.path.append("../wmp_ml/dag")

In [25]:
from generator import DagGenerator
from datetime import datetime


In [26]:
d = {}
d.setdefault('HELLO', {})['newcell'] = 5
d

{'HELLO': {'newcell': 5}}

In [27]:
tt = {
        'test': 'test',
        'args': {
                                                    ('Department', 'EducationField', 'JobRole', "MaritalStatus"): {wmp_get_dummies: None},
                                                    'BusinessTravel': {create_ordinal_df: {'ordinal_names':{"Non-Travel" :0, "Travel_Rarely":1, "Travel_Frequently": 2}}},
                                                    ('Attrition', 'Gender', 'OverTime') : {convert_boolean_df: {'boolean_names_and_values':[["Attrition", "Yes", "No"],
                                                                                                                  ["Gender", "Male", "Female"],
                                                                                                                   ["OverTime", "Yes", "No"]], 
                                                                                                                'tag' : None}},
                                                    ('Age', 'DistanceFromHome', 'EmployeeCount', "EnvironmentSatisfaction",
                                                    'JobInvolvement', 'JobSatisfaction', "MonthlyIncome",
                                                    'NumCompaniesWorked', 'PercentSalaryHike', "PerformanceRating",
                                                    'RelationshipSatisfaction', 'StandardHours', 'TotalWorkingYears',
                                                    'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
                                                    'YearsInCurrentRole', 'YearsSinceLastPromotion',
                                                    'YearsWithCurrManager') : {normalize_values: {'transformer': StandardScaler, 
                                                                                                  'tag': None}},
                                                    'MonthlyRate': {yeojohnson: None,
                                                                    normalize_values: {'transformer': StandardScaler,
                                                                                       'tag': None}},
                                               }}

In [28]:
template = '''\nPythonOperator( 
        task_id={},
        provide_context=True,
        python_callable={},
        params = {},
        dag = dag)\n
'''

def parse_params(params):
    params_str = str(params)
    obj_dict = {}
    for k,v in params.items():
        if any([inspect.isclass(k),
                inspect.ismodule(k),
                inspect.isfunction(k)]):
            obj_dict[str(k)] = k
        elif any([inspect.isclass(v),
                inspect.ismodule(v),
                inspect.isfunction(v)]):
            obj_dict[str(v)] = v
            
        if isinstance(v, dict):
            v = parse_params(v)
    
    for key in obj_dict:
        params_str = params_str.replace(key, obj_dict[key].__name__)
        
    
    
    
    return params_str
        

def test_parser(kwargs):
    for key in kwargs:

        for key2 in kwargs[key]:
            if kwargs[key][key2] == None:
                kwargs[key][key2] = {}
            print()
            print(template.format(key,
				key2.__name__,
                parse_params(kwargs[key][key2])))
            print()
            
#test_parser(tt)

In [29]:
d = {}
d[StandardScaler()] = "HIHI"
list(d.keys())[0].fit

<bound method StandardScaler.fit of StandardScaler(copy=True, with_mean=True, with_std=True)>

In [30]:
dag_args = { 
                'dag_name': "WMP_ML_test_creation_short",
                'dag':      {
                                'owner': 'Sam Showalter',
                                'email': ['sshowalter@wmp.com'],
                                'op_args':{},
                                'op_kwargs': {},
                                'params': {}
                            },
                'config' : kwargs
}

In [31]:
dg = DagGenerator(dag_args)
dg.generate_file()

{'model': <class 'sklearn.linear_model.logistic.LogisticRegression'>, 'params': {}}
{'model': <class 'sklearn.naive_bayes.GaussianNB'>, 'params': {}}
0 ['data_sources'] ds
2.0 ['preprocessing', 'cleansing'] p_cleansing_l0
2.1 ['preprocessing', 'cleansing'] p_cleansing_l1
3 ['feature_engineering', 'transformations'] fe_transformations
4 ['splitting'] s
5 ['modeling', 'models'] m_models
/home/sshowalter/airflow/dags/WMP_ML_test_creation_short_Sam-Showalter_10-16-2019--18.11.23.py


In [32]:
dd ={
    "data_sources": 0,
    "eda": 1,
    "splitting": 2,
    "preprocessing": 3,
    "feature_engineering": 4,
    "modeling": 5,
    "evaluation": 6,
    "visualization": 7,
    "storage": 8
}

In [33]:
def abc():
    return 'a'

b,c = abc()
print(b,c)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# import json

t = {pd.get_dummies.__name__: None,
 'information': "Do not change this"}


# def set_default(obj):
#     if isinstance(obj, set):
#         return list(obj)
#     raise TypeError

json.dumps(t).replace( '"' + pd.get_dummies.__name__ + '"', pd.get_dummies.__name__).replace('null', 'None')

# with open('../repos/WMP_ML_MVP/wmp_ml/config/dag_hierarchy.cfg', 'w') as config:
#     config.write(json.dumps(dd, indent=4))
#     config.close()


In [ ]:
pconfig = {   'splitting': {2: ['train_test_split', 'k_fold']},
             'data_souces': {0: {'csv': 0, 'sql': 1, 'pickle': 2}},
             'preprocessing': {3: {0: 'outliers', 1: 'missing_data'}},
             'evaluation': {6: ['accuracy',
               'f1',
               'confusion_matrix',
               'classification_report',
               'precision',
               'recall']},
             'eda': {1: {'profiling': 0}},
             'modeling': {5: {'hyperparameter_tuning': {}, 'models': {}}},
             'storage': {8: {'model': 'FILEPATH',
               'pipeline': 'FILEPATH',
               'performance': 'FILEPATH',
               'data': 'FILEPATH'}},
             'feature_engineering': "HI",
              
             'visualization': {7: ['AUROC']}}
          
print(pprint.pformat(config))